In [3]:
def torch_cuda_diagnostics():
    try:
        import torch
    except ImportError:
        print("❌ PyTorch is NOT installed in this environment.")
        return

    print(f"✅ PyTorch imported — version: {torch.__version__}")

    if torch.cuda.is_available():
        print("✅ CUDA is available.")
        print(f"   ‣ CUDA runtime (from wheel): {torch.version.cuda}")
        print(f"   ‣ Number of visible GPUs  : {torch.cuda.device_count()}")
        # Grab information about the first device
        dev = torch.cuda.get_device_properties(0)
        print(f"   ‣ GPU 0 name              : {dev.name}")
        print(f"   ‣ Compute capability      : {dev.major}.{dev.minor}")
        print(f"   ‣ Total memory (GiB)      : {dev.total_memory / 2**30:.1f}")
    else:
        # Torch loaded, but either no GPU or wrong wheel (CPU-only build)
        print("⚠️  CUDA NOT available. Possible reasons:")
        print("   • No NVIDIA GPU/drivers detected")
        print("   • Driver too old for the wheel's CUDA version")
        print("   • Installed the CPU-only torch wheel")

if __name__ == "__main__":
    torch_cuda_diagnostics()


✅ PyTorch imported — version: 2.6.0+cu124
✅ CUDA is available.
   ‣ CUDA runtime (from wheel): 12.4
   ‣ Number of visible GPUs  : 1
   ‣ GPU 0 name              : NVIDIA GeForce RTX 4070 Laptop GPU
   ‣ Compute capability      : 8.9
   ‣ Total memory (GiB)      : 8.0


## Download EDF

In [15]:
from pathlib import Path
import pyedflib
import numpy as np

# 1. Build the path safely
edf_path = Path(r"C:\Users\blend\Desktop\CS\hacktech\data\00000995-100507[001].edf")   # raw-string or forward slashes

# 2. (Optional) sanity check before opening
if not edf_path.exists():
    raise FileNotFoundError(edf_path)

# 3. Open as before
with pyedflib.EdfReader(str(edf_path)) as f:
    labels = f.getSignalLabels()
    sigbufs = {lbl: f.readSignal(i) for i, lbl in enumerate(labels)}

print(sigbufs['Tracheal'])
print(sigbufs['Mic'])

[ 0.00273136  0.00343328  0.00294499 ...  0.00178531 -0.0016022
 -0.00218204]
[0.03254749 0.03282216 0.03334096 ... 0.0142977  0.01350423 0.0127718 ]


## Read RML

In [21]:
def guess_rml(path):
    h = open(path, 'rb').read(512)
    try:
        txt = h.decode('latin1')
    except UnicodeDecodeError:
        return "binary"
    if "<documentTemplate" in txt:
        return "reportlab"
    if "<markup" in txt:
        return "cad_redline"
    if "@prefix" in txt and " rr:" in txt:
        return "rdf_mapping"
    return "sleepware" if b"\x00\x00\x00" in h else "unknown"

print(guess_rml("00000995-100507.rml"))


FileNotFoundError: [Errno 2] No such file or directory: '00000995-100507.rml'